# Imports and Configurations

In [1]:
import pandas as pd 
import numpy as np

# Data

The data used is taken from a kaggle data set, scraped from yahoo finance - https://www.kaggle.com/stefanoleone992/mutual-funds-and-etfs?select=Mutual+Funds.csv.

The last update was collected on November 2020. x-year, three year return's for example, statistics refer to x years from Noverember 2020, not 2017-2019 returns. 

In [2]:
mut_funds = pd.read_csv("/content/drive/MyDrive/Projects/MutualFunds/Data/Mutual Funds.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (167,171) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
mut_funds.columns

Index(['fund_symbol', 'fund_extended_name', 'fund_family', 'inception_date',
       'category', 'rating', 'return_rating', 'risk_rating',
       'investment_strategy', 'investment_type',
       ...
       'fund_sharpe_ratio_5years', 'category_sharpe_ratio_5years',
       'fund_sharpe_ratio_10years', 'category_sharpe_ratio_10years',
       'fund_treynor_ratio_3years', 'category_treynor_ratio_3years',
       'fund_treynor_ratio_5years', 'category_treynor_ratio_5years',
       'fund_treynor_ratio_10years', 'category_treynor_ratio_10years'],
      dtype='object', length=173)

In [4]:
# Unique mutual funds features (removing quartely, yearly, etc groups)

mut_cols = mut_funds.columns
mut_unq_cols = mut_cols.str.split("_").map(lambda x : "_".join(x[0:2]) if len(x) == 3 else "_".join(x[0:3])).drop_duplicates()
mut_unq_cols.sort_values()

Index(['asset_bonds', 'asset_cash', 'asset_convertable', 'asset_others',
       'asset_preferred', 'asset_stocks', 'bond_duration', 'bond_maturity',
       'category', 'category_alpha', 'category_beta', 'category_mean_annual',
       'category_net_annual', 'category_r_squared', 'category_return',
       'category_sharpe_ratio', 'category_standard_deviation',
       'category_treynor_ratio', 'credit_a', 'credit_aa', 'credit_aaa',
       'credit_b', 'credit_bb', 'credit_bbb', 'credit_below', 'credit_other',
       'credit_us', 'currency', 'fund_alpha', 'fund_beta', 'fund_extended',
       'fund_family', 'fund_mean_annual', 'fund_net_annual', 'fund_r_squared',
       'fund_return', 'fund_return_2010', 'fund_return_2011',
       'fund_return_2012', 'fund_return_2013', 'fund_return_2014',
       'fund_return_2015', 'fund_return_2016', 'fund_return_2017',
       'fund_return_2018', 'fund_return_2019', 'fund_return_2020',
       'fund_sharpe_ratio', 'fund_standard_deviation', 'fund_symbol',
 

In [5]:
s = mut_funds.isna().sum()
s[s>0].sort_values(ascending=False)

credit_us_government                 14789
credit_aaa                           14789
credit_aa                            14789
credit_a                             14789
credit_bbb                           14789
                                     ...  
category_alpha_3years                   10
category_net_annual_expense_ratio       10
category_treynor_ratio_10years          10
fund_yield                               3
net_asset_value                          2
Length: 164, dtype: int64

# Further look into similar groups of features

## General characteristics features



In [6]:
general = mut_cols[:12]
general

Index(['fund_symbol', 'fund_extended_name', 'fund_family', 'inception_date',
       'category', 'rating', 'return_rating', 'risk_rating',
       'investment_strategy', 'investment_type', 'size_type', 'currency'],
      dtype='object')

In [7]:
mut_funds[general].sample()

,fund_symbol,fund_extended_name,fund_family,inception_date,category,rating,return_rating,risk_rating,investment_strategy,investment_type,size_type,currency
9515,HIMYX,Pioneer High Income Municipal Fund Class Y,Pioneer Investments,2006-10-15,High Yield Muni,5.0,4.0,3.0,The investment seeks to maximize total return ...,Blend,Small,USD


In [8]:
mut_funds[general].isna().sum()

fund_symbol               0
fund_extended_name        0
fund_family               0
inception_date            0
category                 10
rating                 1040
return_rating          1072
risk_rating            1040
investment_strategy       0
investment_type        1195
size_type              1195
currency                  0
dtype: int64

In [9]:
mut_funds[general].nunique()

fund_symbol            24821
fund_extended_name     24804
fund_family              722
inception_date          4404
category                 113
rating                     5
return_rating              5
risk_rating                5
investment_strategy     7095
investment_type            3
size_type                  3
currency                   1
dtype: int64

In [10]:
general_red = general.drop(['currency','investment_strategy','fund_extended_name','fund_family','category','fund_symbol'])
general_red

Index(['inception_date', 'rating', 'return_rating', 'risk_rating',
       'investment_type', 'size_type'],
      dtype='object')

## Fund Statistics

In [11]:
s = mut_cols[mut_cols.str.startswith('fund')]
fund = s[~s.isin(general)]
fund

Index(['fund_net_annual_expense_ratio', 'fund_yield', 'fund_return_ytd',
       'fund_return_1month', 'fund_return_3months', 'fund_return_1year',
       'fund_return_3years', 'fund_return_5years', 'fund_return_10years',
       'fund_return_2019', 'fund_return_2018', 'fund_return_2017',
       'fund_return_2016', 'fund_return_2015', 'fund_return_2014',
       'fund_return_2013', 'fund_return_2012', 'fund_return_2011',
       'fund_return_2010', 'fund_return_2020_q3', 'fund_return_2020_q2',
       'fund_return_2020_q1', 'fund_return_2019_q4', 'fund_return_2019_q3',
       'fund_return_2019_q2', 'fund_return_2019_q1', 'fund_return_2018_q4',
       'fund_return_2018_q3', 'fund_return_2018_q2', 'fund_return_2018_q1',
       'fund_return_2017_q4', 'fund_return_2017_q3', 'fund_return_2017_q2',
       'fund_return_2017_q1', 'fund_return_2016_q4', 'fund_return_2016_q3',
       'fund_return_2016_q2', 'fund_return_2016_q1', 'fund_return_2015_q4',
       'fund_return_2015_q3', 'fund_return_2015_q2

In [12]:
#Three years are at the time of data gathering, not the previous three years 

temp = mut_funds[['fund_return_2019', 'fund_return_2018', 'fund_return_2017']].sum(axis=1)
(temp - mut_funds.fund_return_3years)[:3]

0    27.70
1    23.29
2    41.35
dtype: float64

In [13]:
mut_funds.fund_return_3years

0         3.03
1         9.71
2        19.44
3         6.79
4         3.04
         ...  
24816    -5.61
24817    39.68
24818    39.38
24819    39.15
24820    39.47
Name: fund_return_3years, Length: 24821, dtype: float64

## Sector Percentages 

In [14]:
sector_perc = mut_cols[mut_cols.str.startswith('sector')]
sector_perc

Index(['sector_basic_materials', 'sector_consumer_cyclical',
       'sector_financial_services', 'sector_real_estate',
       'sector_consumer_defensive', 'sector_healthcare', 'sector_utilities',
       'sector_communication_services', 'sector_energy', 'sector_industrials',
       'sector_technology'],
      dtype='object')

In [15]:
mut_funds[sector_perc].sample()

,sector_basic_materials,sector_consumer_cyclical,sector_financial_services,sector_real_estate,sector_consumer_defensive,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology
22129,2.75,9.54,12.16,7.8,10.49,13.68,2.8,8.38,4.4,10.22,17.77


In [16]:
mut_funds[sector_perc].sample().sum(axis=1)

15132    100.0
dtype: float64

In [17]:
mut_funds[sector_perc].isna().sum()

sector_basic_materials           5790
sector_consumer_cyclical         5790
sector_financial_services        5790
sector_real_estate               5790
sector_consumer_defensive        5790
sector_healthcare                5790
sector_utilities                 5790
sector_communication_services    5790
sector_energy                    5790
sector_industrials               5790
sector_technology                5790
dtype: int64

## Asset Percentages 


In [18]:
asset_perc = mut_cols[mut_cols.str.startswith('asset')]
asset_perc

Index(['asset_cash', 'asset_stocks', 'asset_bonds', 'asset_others',
       'asset_preferred', 'asset_convertable'],
      dtype='object')

In [19]:
mut_funds[asset_perc].sample()

,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertable
3534,6.8,0.0,89.25,0.0,0.0,3.95


In [20]:
mut_funds[asset_perc].isna().sum()

asset_cash           27
asset_stocks         27
asset_bonds          27
asset_others         27
asset_preferred      27
asset_convertable    27
dtype: int64

## Credit Percentages

In [21]:
credit_perc = mut_cols[mut_cols.str.startswith('credit')]
credit_perc

Index(['credit_us_government', 'credit_aaa', 'credit_aa', 'credit_a',
       'credit_bbb', 'credit_bb', 'credit_b', 'credit_below_b',
       'credit_other_ratings'],
      dtype='object')

In [22]:
mut_funds[credit_perc].sample()

,credit_us_government,credit_aaa,credit_aa,credit_a,credit_bbb,credit_bb,credit_b,credit_below_b,credit_other_ratings
17068,0.0,10.51,38.56,34.59,14.31,0.5,0.0,0.31,1.22


In [23]:
mut_funds[credit_perc].isna().sum()

credit_us_government    14789
credit_aaa              14789
credit_aa               14789
credit_a                14789
credit_bbb              14789
credit_bb               14789
credit_b                14789
credit_below_b          14789
credit_other_ratings    14789
dtype: int64

In [24]:
mut_funds.credit_us_government.value_counts()

0.0    10032
Name: credit_us_government, dtype: int64

## Price Percentages

In [25]:
price_perc = mut_cols[mut_cols.str.startswith('price')]
price_perc

Index(['price_earnings_ratio', 'price_book_ratio', 'price_sales_ratio',
       'price_cashflow_ratio'],
      dtype='object')

In [26]:
mut_funds[price_perc].sample()

,price_earnings_ratio,price_book_ratio,price_sales_ratio,price_cashflow_ratio
12123,22.68,2.55,1.8,13.11


In [27]:
mut_funds[price_perc].isna().sum()

price_earnings_ratio    6965
price_book_ratio        6330
price_sales_ratio       5729
price_cashflow_ratio    6503
dtype: int64

## Other

In [28]:
cond1 = ~mut_cols.isin(price_perc.union(fund).union(general).union(sector_perc).union(asset_perc).union(credit_perc)) #Dont know why I can't pass an array of these objects to union
cond2 = ~mut_cols.str.startswith('category')

other = mut_cols[cond1 & cond2]
other

Index(['median_market_cap', 'bond_maturity', 'bond_duration',
       'net_asset_value', 'top10_holdings', 'years_up', 'years_down',
       'quarters_up', 'quarters_down'],
      dtype='object')

# Data Cleaning

## Initial trimming of data

Note:

* Not including any of the category variables because for them to be useful need to perform additional feature engineering to (ie new_var = fund_alpha - category_alpha)

* Not including any of the asset, credit, or sector percentages because too many missing values. Need to impute, or determine why missing. 




In [29]:
cols_reduced = pd.Index(['fund_symbol','inception_date',
            'fund_net_annual_expense_ratio','fund_yield',
            'fund_return_3years','fund_return_5years', 'fund_return_10years',
            'fund_r_squared_3years', 'fund_r_squared_5years','fund_r_squared_10years', 
            'fund_standard_deviation_3years','fund_standard_deviation_5years', 'fund_standard_deviation_10years',
            'fund_sharpe_ratio_3years', 'fund_sharpe_ratio_5years','fund_sharpe_ratio_10years', 
            'fund_treynor_ratio_3years','fund_treynor_ratio_5years', 'fund_treynor_ratio_10years',
            'fund_alpha_3years', 'fund_alpha_5years','fund_alpha_10years',
            'fund_beta_3years', 'fund_beta_5years','fund_beta_10years', 
            'fund_mean_annual_return_3years', 'fund_mean_annual_return_5years', 'fund_mean_annual_return_10years',
            'rating','risk_rating','return_rating',
            'quarters_up','quarters_down',
            'investment_type','size_type','median_market_cap','net_asset_value','fund_return_ytd'])

In [30]:
data = mut_funds[cols_reduced]

In [31]:
data.head()

,fund_symbol,inception_date,fund_net_annual_expense_ratio,fund_yield,fund_return_3years,fund_return_5years,fund_return_10years,fund_r_squared_3years,fund_r_squared_5years,fund_r_squared_10years,fund_standard_deviation_3years,fund_standard_deviation_5years,fund_standard_deviation_10years,fund_sharpe_ratio_3years,fund_sharpe_ratio_5years,fund_sharpe_ratio_10years,fund_treynor_ratio_3years,fund_treynor_ratio_5years,fund_treynor_ratio_10years,fund_alpha_3years,fund_alpha_5years,fund_alpha_10years,fund_beta_3years,fund_beta_5years,fund_beta_10years,fund_mean_annual_return_3years,fund_mean_annual_return_5years,fund_mean_annual_return_10years,rating,risk_rating,return_rating,quarters_up,quarters_down,investment_type,size_type,median_market_cap,net_asset_value,fund_return_ytd
0,AAAAX,2007-07-29,1.22,1.22,3.03,4.08,2.38,85.80,78.81,77.74,12.46,10.39,8.67,0.18,0.33,0.24,1.32,2.77,1.91,-2.51,-2.86,-3.86,1.09,1.05,0.93,0.31,0.38,0.23,3.0,3.0,3.0,27,15,Blend,Large,18592.11,6.979100e+08,-7.60
1,AAADX,2011-12-29,1.43,2.80,9.71,8.46,10.88,87.23,84.28,89.77,8.46,10.21,11.70,0.93,0.74,0.89,6.82,5.38,7.1,0.17,-0.30,-0.15,1.17,1.36,1.45,0.80,0.72,0.92,4.0,5.0,5.0,27,5,Blend,Large,168478.91,8.938000e+07,0.30
2,AAAGX,1999-10-28,1.12,0.00,19.44,15.26,14.71,90.82,85.84,87.04,19.81,16.49,15.40,0.92,0.88,0.93,16.87,13.48,13.03,7.68,2.92,0.77,1.06,1.04,1.08,1.65,1.30,1.25,3.0,4.0,3.0,31,12,Growth,Large,340488.16,1.490000e+09,24.20
3,AAAIX,2000-07-31,0.63,1.77,6.79,8.19,8.66,98.48,97.71,97.20,14.93,12.35,11.44,0.41,0.61,0.73,3.7,5.05,5.87,0.07,-0.78,-0.48,1.41,1.39,1.37,0.64,0.72,0.75,4.0,3.0,4.0,34,8,Blend,Large,29418.97,7.072500e+08,3.17
4,AAANX,2012-01-30,1.45,0.69,3.04,5.73,NaN,95.21,94.66,NaN,18.26,15.06,NaN,0.17,0.37,NaN,0.86,2.73,NaN,-4.19,-4.38,NaN,1.69,1.67,NaN,0.39,0.56,NaN,3.0,5.0,4.0,26,8,Blend,Large,56230.30,4.955100e+08,-3.26


In [32]:
data.isna().sum()

fund_symbol                           0
inception_date                        0
fund_net_annual_expense_ratio         0
fund_yield                            3
fund_return_3years                  944
fund_return_5years                 2530
fund_return_10years                7551
fund_r_squared_3years               904
fund_r_squared_5years              2496
fund_r_squared_10years             7527
fund_standard_deviation_3years      904
fund_standard_deviation_5years     2496
fund_standard_deviation_10years    7527
fund_sharpe_ratio_3years            904
fund_sharpe_ratio_5years           2496
fund_sharpe_ratio_10years          7527
fund_treynor_ratio_3years           904
fund_treynor_ratio_5years          2496
fund_treynor_ratio_10years         7527
fund_alpha_3years                   904
fund_alpha_5years                  2496
fund_alpha_10years                 7527
fund_beta_3years                    904
fund_beta_5years                   2496
fund_beta_10years                  7527


**In order to include new funds, drop the 5 and 10 year old statistics**, also have to remove quarters up/down because older funds obviously will have more high/low quarters

In [33]:
searchfor = ['5years', '10years','quarters']
cols_reduced = cols_reduced[~cols_reduced.str.contains('|'.join(searchfor))]
cols_reduced

Index(['fund_symbol', 'inception_date', 'fund_net_annual_expense_ratio',
       'fund_yield', 'fund_return_3years', 'fund_r_squared_3years',
       'fund_standard_deviation_3years', 'fund_sharpe_ratio_3years',
       'fund_treynor_ratio_3years', 'fund_alpha_3years', 'fund_beta_3years',
       'fund_mean_annual_return_3years', 'rating', 'risk_rating',
       'return_rating', 'investment_type', 'size_type', 'median_market_cap',
       'net_asset_value', 'fund_return_ytd'],
      dtype='object')

In [34]:
data = data[cols_reduced]

In [35]:
print(data.shape, data.dropna().shape) # Little information lost, only the funds less than 3 years old

(24821, 20) (22692, 20)


In [36]:
data.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


# Data Cleaning

In [37]:
data.dtypes[data.dtypes == 'object']

fund_symbol                  object
inception_date               object
fund_treynor_ratio_3years    object
investment_type              object
size_type                    object
dtype: object

In [38]:
# Alpha and Treyner contain values with commas 

data.fund_alpha_3years = pd.to_numeric(data.fund_alpha_3years.replace(',',''))
data.fund_treynor_ratio_3years = pd.to_numeric(data.fund_treynor_ratio_3years,errors='coerce')

data = data[~data.fund_treynor_ratio_3years.isna()]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [39]:
data.dtypes[data.dtypes == 'object']

fund_symbol        object
inception_date     object
investment_type    object
size_type          object
dtype: object

In [41]:
data.to_csv("/content/drive/MyDrive/Projects/MutualFunds/Data/MutualFunds_Cleaned.csv",index=False)